In [3]:
df = pd.read_csv('CleanedDataNew.csv')

In [2]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from string import punctuation,digits
import re
from keras.utils import to_categorical
#from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#from pandas_ml import ConfusionMatrix
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Embedding
# from keras.utils.np_utils import probas_to_classes
from sklearn import preprocessing
from keras.models import Sequential
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib import rcParams
from random import shuffle
# figure size in inches
rcParams['figure.figsize'] = 11.7,8.27
#import word2vecReader as godin_embedding
#from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
#import skopt
#from skopt import gp_minimize, forest_minimize
#from skopt.space import Real, Categorical, Integer
#from skopt.utils import use_named_args


In [7]:
df.head()
df_new= df.drop(columns=['url'])

In [8]:
df_new.head()


,text,target
0,Century-old mud houses amidst concrete jungle:...,0
1,Group of ministers formed to unlock land of cl...,0
2,Dacoits batter infant in Garia: Seven armed da...,1
3,TMC warns workers against sand mining: A day a...,0
4,'Policymakers should have bigger vision': A se...,0


In [60]:
len(df_new['text'])

6326

In [9]:
def remove_punctuation(s):
    list_punctuation = list(punctuation)
    for i in list_punctuation:
        s = s.replace(i,' ')
    return s

In [10]:
def clean_sentence(sentence):
    sentence = sentence.lower()
    #remove multiple repeat non num-aplha char !!!!!!!!!-->!
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
#     print(sentence)
    #removes alpha char repeating more than twice aaaa->aa
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
#     print(sentence)
    #removes links
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', r'', sentence)
#     print(sentence)
    # remove @usernames
    sentence = re.sub(r"\@(\w+)", "", sentence)
#     print(sentence)
    #removing stock names to see if it helps
#     sentence = re.sub(r"(?:\$|https?\://)\S+", "", sentence)
    #remove # from #tags
    sentence = sentence.replace('#','')
    sentence = sentence.replace("'s",'')
    sentence = sentence.replace("-",' ')
#     print(sentence)
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    tokens = [remove_punctuation(w) for w in tokens]
#     print(tokens)
    #     remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
#no removing non alpha words to keep stock names($ZSL)
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
#     for w in stop_words:
#         print(w)
#     print(tokens)
    # filter out short tokens
#     tokens = [word for word in tokens if len(word) > 1]
#     print(tokens)
    remove_digits = str.maketrans('', '', digits)
#     print(tokens)
    tokens = [w.translate(remove_digits) for w in tokens]
    tokens = [w.strip() for w in tokens]
    tokens = [w for w in tokens if w!=""]
#     print(tokens)
    tokens = ' '.join(tokens)
    return tokens

In [11]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [12]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [13]:
#not needed here since this is binary classification 
def convert_lables(trainY,testY):
    le = preprocessing.LabelEncoder()
    le.fit(trainY+testY)
    temp1 = le.transform(trainY)
    return to_categorical(temp1,no_of_classes),le.classes_

In [42]:
#loading GloVe embedding
def load_GloVe_embedding(file_name):
    embeddings_index = dict()
    f = open(file_name,encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

In [15]:
# create a weight matrix for words in training docs
def get_GloVe_embedding_matrix(embeddings_index):
    embedding_matrix = np.zeros((vocab_size, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [22]:
sentence= df_new['text']
labels  = df_new['target']

In [25]:
sentence = [clean_sentence(x) for x in sentence]

In [26]:
lengths = [len(s.split()) for s in sentence]

In [51]:
print('max len = ',max(lengths))
#sns.distplot(lengths)


max len =  20


In [32]:
max_length  = max(lengths)
X = sentence
Y = labels

In [34]:
tokenizer = create_tokenizer(X)
vocab_size = len(tokenizer.word_index) + 1
trainX = encode_text(tokenizer, X, max_length)
# testX = encode_text(tokenizer, testX, max_length)
#trainY,lable_encoding = convert_lables(Y,[])

In [65]:
print(vocab_size)
print(len(trainX))

print(len(X))

15207
6326
6326


In [43]:
embeddings_index_glove = load_GloVe_embedding('glove.6B.100d.txt')

Loaded 400000 word vectors.


In [56]:
len(embeddings_index_glove)

400000

In [47]:
get_embeddings = get_GloVe_embedding_matrix(embeddings_index_glove)

In [73]:
#embeddings for each word existing in the given dataset
get_embeddings[1000]

array([-0.018118  ,  0.28816   ,  0.52890003,  0.64172   ,  0.14843   ,
        0.081948  , -0.65104002,  0.83174002,  0.17551   , -0.53397   ,
       -0.26334   ,  0.17964   , -0.12782   ,  0.06143   , -0.46009001,
       -0.68702   ,  0.2938    , -0.29495999,  0.45078   ,  1.12720001,
       -0.10113   ,  0.31600001, -0.97586   , -0.21569   ,  0.034138  ,
       -0.37158999, -0.85093999, -0.49879   ,  0.020042  , -1.39919996,
       -0.061732  ,  0.96856999,  0.055472  ,  0.042682  ,  0.051694  ,
        1.06620002, -0.17106   , -0.35253999, -0.43452999, -0.12687001,
        0.75879002,  0.46337   , -0.16287   , -0.37125999, -0.45442   ,
       -0.24525   , -0.96293002, -0.32804999, -0.025023  , -0.013757  ,
       -0.067559  , -0.31332001, -0.14564   ,  0.47632   , -0.82901001,
       -0.78926003,  0.25163999, -0.66337001,  1.13100004, -0.24765   ,
        1.23829997, -0.65959001, -0.028934  , -0.85433   ,  0.21554001,
        0.002759  ,  0.24817   ,  0.11609   ,  0.40447   ,  0.48

In [62]:
#For each sentence
#1. See all words's emebeddings (of size 100 for each word)
#2. Take the mean of A[0] ,B[0], C[0].....N[0], such that A, B,C .. N are words of embeddings.
#3. Do this till A[100]...N[100]
#4. This creates a vector sentence1 : [mean of first bit of  words........... mean of last bit of words]


In [81]:
#Finds the mean of all the words in the sentence
def mean_vector_sentence(sentence):
    l1=[]
    for word in sentence.split():
        l1.append(get_embeddings[tokenizer.word_index[word]])
    mat = np.array(l1)
    v=np.mean(mat)
    return v
        
    

In [82]:
#Each sentence is now a vector , taken by mean of vectors of the words in the sentence
vect_sentence=[]
for s in sentence:
    vect_sentence.append(mean_vector_sentence(s))

In [85]:
len(vect_sentence)
len(Y)

6326

In [86]:
#Now for example title for each title will be created such.
#Will content embeddings be done sentence wise or the entire row content into  one vector
#These embeddings will be further used in the cnn network for the fused main model.

In [1]:
embedding_dim = 256
filter_sizes =4
num_filters = 512
drop = 0.4

epochs = 100
batch_size = 10

In [2]:
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv_Title = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
    
# One more  layer for content(1)
# Fuse content and knowledge graph layer(2)
# One more layer for image(3)
# Fuse (1) (2) (3)

IndentationError: unindent does not match any outer indentation level (<ipython-input-2-377eace80196>, line 5)

In [ ]:
# define the model
def define_model(length, vocab_size):
	# channel 1 : title
	inputs1 = Input(shape=(length,))
    #embedding dimention :256
	embedding1 = Embedding(vocab_size,100)(inputs1)
    #Filter size:4 
	#conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    #This is what I think it should be,becuase embedding dimention is256
    conv2= Conv2D(num_filters, kernel_size=(4,256), padding='valid', kernel_initializer='normal', activation='relu')(embedding1)
	#drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(conv2)
	#pool1 = MaxPooling2D(pool_size=2)(conv2)
	flat1 = Flatten()(pool1)
    
    
	# channel 2 :  CONTENT + KG
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	#drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3 #IMAGES
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='final.png')
	return model